# Daten einlesen

## Notwendige Bibliotheken importieren

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import numpy as np
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

## Daten laden und in DataFrame verbinden

In [ ]:
# Download Data
umsatzdaten_url = "https://github.com/opencampus-sh/einfuehrung-in-data-science-und-ml/raw/main/umsatzdaten_gekuerzt.csv"

kiwodaten_url = "https://github.com/opencampus-sh/einfuehrung-in-data-science-und-ml/raw/main/kiwo.csv"

wetterdaten_url = "https://github.com/opencampus-sh/einfuehrung-in-data-science-und-ml/raw/main/wetter.csv"

# Daten von den URLs herunterladen und in Pandas DataFrames laden
umsatzdaten = pd.read_csv(umsatzdaten_url)
kiwo = pd.read_csv(kiwodaten_url)
wetter = pd.read_csv(wetterdaten_url)

#Outer Join
merged_data = pd.merge(umsatzdaten, kiwo, on='Datum', how='outer')
merged_data = pd.merge(merged_data, wetter, on='Datum', how='outer')

# Überprüfen, ob `KielerWoche` NaN-Werte korrekt darstellt
print(merged_data['KielerWoche'])

# Gib den zusammengeführten DataFrame aus
print(merged_data)

# Überprüfung auf fehlende Werte

In [ ]:
# Überprüfung auf fehlende Werte pro Spalte
missing_values = merged_data.isnull()
print("Anzahl fehlender Werte pro Spalte:")
print(missing_values.sum())

# Gesamte Anzahl fehlender Werte
total_missing_values = missing_values.sum().sum()
print("\nGesamte Anzahl fehlender Werte:")
print(total_missing_values)




## Datum konvertieren / Wochentag / Feiertage hinzufügen

In [ ]:
#Konvertiere die Spalte "Datum" in einen Datumsdatentypen 
merged_data["Datum"] = pd.to_datetime(merged_data["Datum"])

# Hinzufügen einer Spalte für den numerischen Wochentag (Montag=0, Sonntag=6)
merged_data['Wochentag_Numerisch'] = merged_data['Datum'].dt.weekday

# Überprüfen, ob `KielerWoche` NaN-Werte korrekt darstellt und Umwandlung in binären Wert
merged_data['KielerWoche'] = merged_data['KielerWoche'].notna().astype(int)

# Hinzufügen einer Spalte für Wochenenden (Samstag und Sonntag)
merged_data['Wochenende'] = (merged_data['Datum'].dt.dayofweek >= 5).astype(int)

# Feiertage als Liste von Datumsangaben
feiertage = pd.to_datetime([
    '2012-01-01', '2012-04-06', '2012-04-09', '2012-05-01', '2012-05-17', '2012-05-28', '2012-10-03', '2012-12-25', '2012-12-26', '2013-01-01', '2013-03-29', '2013-04-01', '2013-05-01', '2013-05-09', '2013-05-20', '2013-10-03', '2013-12-25', '2013-12-26', '2014-01-01', '2014-04-18', '2014-04-21', '2014-05-01', '2014-05-29', '2014-06-09', '2014-10-03', '2014-12-25', '2014-12-26', '2015-01-01', '2015-04-03', '2015-04-06', '2015-05-01', '2015-05-14', '2015-05-25', '2015-10-03', '2015-12-25', '2015-12-26', '2016-01-01', '2016-03-25', '2016-03-28', '2016-05-01', '2016-05-05', '2016-05-16', '2016-10-03', '2016-12-25', '2016-12-26', '2017-01-01', '2017-04-14', '2017-04-17', '2017-05-01', '2017-05-25', '2017-06-05', '2017-10-03', '2017-12-25', '2017-12-26',
'2018-01-01', '2018-03-30', '2018-04-02', '2018-05-01', '2018-05-10', '2018-05-21', '2018-10-03', '2018-12-25', '2018-12-26', '2019-01-01', '2019-04-19', '2019-04-22', '2019-05-01', '2019-05-30', '2019-06-10', '2019-10-03', '2019-12-25', '2019-12-26'
] )

#Füge eine Spalte hinzu, die angibt, ob das Datum ein Feiertag ist (1) oder nicht (0)
merged_data['Feiertag'] = merged_data['Datum'].isin(feiertage).astype(int)

# Lagged variable für Feiertage (1 Tag vorher)
merged_data['TagVorherFeiertag'] = merged_data['Feiertag'].shift(1).fillna(0).astype(int)

# Lagged variable für Wochenende (1 Tag vorher)
merged_data['TagVorherWochenende'] = merged_data['Wochenende'].shift(1).fillna(0).astype(int)

# Lagged variable für den Umsatz im Vergleich zum Vortag
merged_data['UmsatzVortag'] = merged_data['Umsatz'].shift(1).fillna(0)

## Aggregation des Umsatzes nach Warengruppe und Feiertag / nach Wochentag

In [ ]:
# Aggregation des Umsatzes nach Warengruppe und Feiertag
umsatz_nach_warengruppe_feiertag = merged_data.groupby(['Warengruppe', 'Feiertag'])['Umsatz'].sum().reset_index()
print(umsatz_nach_warengruppe_feiertag)

# Aggregation des Umsatzes nach Wochentag und Warengruppe
umsatz_nach_wochentag_warengruppe = merged_data.groupby(['Wochentag_Numerisch', 'Warengruppe'])['Umsatz'].sum().reset_index()
print(umsatz_nach_wochentag_warengruppe)

### Visualisierung

In [ ]:
# Erstelle die Visualisierung
plt.figure(figsize=(10, 6))
for warengruppe in umsatz_je_wochentag_warengruppe["Warengruppe"].unique():
    data = umsatz_je_wochentag_warengruppe[umsatz_je_wochentag_warengruppe["Warengruppe"] == warengruppe]
    plt.plot(data["Wochentag_Numerisch"], data["Umsatz"], marker='o', label=warengruppe)

plt.xlabel("Wochentag_Numerisch")
plt.ylabel("Umsatz")
plt.title("Umsatz je Wochentag und Warengruppe")
plt.legend(title="Warengruppe")
plt.xticks(rotation=45)

# Deskriptive Statistiken berechnen

In [ ]:
# Temperatur
temp_stats = merged_data['Temperatur'].describe()

# Umsatz
umsatz_stats = merged_data['Umsatz'].describe()

# Wind
wind_stats = merged_data['Windgeschwindigkeit'].describe()

# Umsatz je Warengruppe
deskriptive_statistiken = merged_data.groupby("Warengruppe")["Umsatz"].describe()

## Output deskriptive Statistiken

In [ ]:
# Output der Deskriptiven Statistiken
print("Deskriptive Statistiken für Temperatur:")
print(temp_stats)

print("\nDeskriptive Statistiken für Umsatz:")
print(umsatz_stats)

print("\nDeskriptive Statistiken für Windgeschwindigkeit:")
print(wind_stats)

# Ausgabe der deskriptiven Statistiken
print(deskriptive_statistiken)

# Korrelationen berechnen

In [ ]:
# Korrelationen berechnen
corr_umsatz_weekend = merged_data['Umsatz'].corr(merged_data['Wochenende'].astype(int))
corr_umsatz_weekday = merged_data.groupby('Wochentag_Numerisch')['Umsatz'].mean()
corr_umsatz_temp = merged_data['Umsatz'].corr(merged_data['Temperatur'])
corr_umsatz_wind = merged_data['Umsatz'].corr(merged_data['Windgeschwindigkeit'])

# Korrelationsmatrix für Umsatzdaten
corr_matrix = merged_data.corr()

# Korrelationen zwischen Umsatz und einzelnen Wochentagen berechnen
corr_umsatz_weekdays = {}
for day in range(7):
    # Filtere den DataFrame nach dem aktuellen Wochentag
    day_data = merged_data[merged_data['Wochentag_Numerisch'] == day]
    # Berechne die Korrelation zwischen Umsatz und dem aktuellen Wochentag
    corr = day_data['Umsatz'].corr(day_data['Wochentag_Numerisch'])
    # Speichere die Korrelation für den aktuellen Wochentag
    corr_umsatz_weekdays[day] = corr

# Ausgabe der Korrelationen für die einzelnen Wochentage
for day, corr in corr_umsatz_weekdays.items():
    print(f"Korrelation zwischen Umsatz und {day}: {corr}")

# Berechne die Kontingenztabelle
kontingenztabelle = pd.crosstab(merged_data["Warengruppe"], merged_data["Wochenende"])

# Berechne Cramér's V
chi2 = stats.chi2_contingency(kontingenztabelle)[0]
n = kontingenztabelle.sum().sum()
min_dim = min(kontingenztabelle.shape) - 1
cramers_v = np.sqrt(chi2 / (n * min_dim))

print(f"Cramér's V: {cramers_v}")

# Optional: Ausgabe der Kontingenztabelle zur Überprüfung
print(kontingenztabelle)

# Lineares Modelle

## lineares Modell 1

In [ ]:
# Fit the linear model including categorical variables and additional features
mod = smf.ols('Umsatz ~ C(Warengruppe) + Temperatur + Wochenende + KielerWoche + C(Wochentag_Numerisch)  ', data=merged_data).fit()

# Print the summary
print(mod.summary())

## lineares modell 2

In [ ]:
mod = smf.ols('Umsatz ~ C(Warengruppe) + Temperatur + Wochenende + KielerWoche  + C(Wochentag_Numerisch)', data=merged_data).fit()

# Print the summary
print(mod.summary())



## lineares modell 3

In [ ]:
# Baue das Modell, inklusive Interaktionsterm zwischen Warengruppe und Wochentag
mod = smf.ols('Umsatz ~ C(Warengruppe) * C(Wochentag_Numerisch) + Temperatur + Wochenende + KielerWoche', data=merged_data).fit()

# Print the summary
print(mod.summary())

## lineares modell 4

In [ ]:
# Baue das Modell, inklusive Interaktionstermen zwischen Warengruppe und Wochentag sowie Warengruppe und KielerWoche
mod = smf.ols('Umsatz ~ C(Warengruppe) * C(Wochentag_Numerisch) + Temperatur + Wochenende + C(Warengruppe) * KielerWoche', data=merged_data).fit()

# Print the summary
print(mod.summary())

## lineares modell 5

In [ ]:
# Fit the linear model including categorical variables and additional features
mod = smf.ols('Umsatz ~ C(Warengruppe) + Temperatur + Wochenende + KielerWoche + C(Wochentag_Numerisch)  ', data=merged_data).fit()

# Print the summary
print(mod.summary())

## lineares modell 6 -> bestes Ergebnis bisher

In [ ]:
# Modell inklusive Interaktionen zwischen Warengruppe und Wochentag sowie Warengruppe und KielerWoche
# Zusätzlich Interaktion zwischen lagged variables und Warengruppe hinzugefügt
mod = smf.ols('Umsatz ~ C(Warengruppe) * C(Wochentag_Numerisch) + Temperatur + Wochenende + C(Warengruppe) * KielerWoche + TagVorherFeiertag * C(Warengruppe) + TagVorherWochenende * C(Warengruppe) + UmsatzVortag * C(Warengruppe)', data=merged_data).fit()

# Modellzusammenfassung anzeigen
print(mod.summary())

# Aufteilung der Daten

In [ ]:
# Aufteilen des Datensatzes in Trainings- und Validierungsdaten (70:30)
train_data, validation_data = train_test_split(merged_data, test_size=0.3, random_state=42)


# Training des Modell

## Fehlende Werte 

In [ ]:
# Anzeige der Anzahl der fehlenden Werte in jeder Spalte
missing_values = merged_data.isnull().sum()
print(missing_values)

# Anzeige der Gesamtanzahl der fehlenden Werte
total_missing_values = merged_data.isnull().sum().sum()
print(f"Gesamtanzahl der fehlenden Werte: {total_missing_values}")

In [ ]:
# Imputation mit dem Mittelwert für numerische Spalten im Trainingsdatensatz
train_data_imputed = train_data.copy()
numerische_spalten = train_data.select_dtypes(include=np.number).columns
train_data_imputed[numerische_spalten] = train_data[numerische_spalten].fillna(train_data[numerische_spalten].mean())

# Imputation mit dem Mittelwert für numerische Spalten im Trainingsdatensatz
train_data_imputed = train_data.copy()
numerische_spalten = train_data.select_dtypes(include=np.number).columns
train_data_imputed[numerische_spalten] = train_data[numerische_spalten].fillna(train_data[numerische_spalten].mean())


In [ ]:
# Modellbildung mit Statsmodels
formula = 'Umsatz ~ C(Warengruppe) * C(Wochentag_Numerisch) + Temperatur + Wochenende + C(Warengruppe) * KielerWoche + TagVorherFeiertag * C(Warengruppe) + TagVorherWochenende * C(Warengruppe) + UmsatzVortag * C(Warengruppe)'
mod = smf.ols(formula, data=train_data_imputed).fit()

# Modellzusammenfassung anzeigen
print(mod.summary())

## Modell trainieren